1. take only the main text -> no label -> zero shot 20% of them 
2. use these labels for training logistic regression model
3. use that logistic regression model for the active learning cycle

## zero shot and log reg

In [2]:
import pandas as pd
from textblob import TextBlob
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report


In [3]:
df = pd.read_csv('/Users/sriyan/Documents/techjam/sentiment-analysis/data/sentimentdataset.csv')
df.head()

,Unnamed: 0.1,Unnamed: 0,Text,Sentiment,Timestamp,User,Platform,Hashtags,Retweets,Likes,Country,Year,Month,Day,Hour
0,0,0,Enjoying a beautiful day at the park! ...,Positive,2023-01-15 12:30:00,User123,Twitter,#Nature #Park,15.0,30.0,USA,2023,1,15,12
1,1,1,Traffic was terrible this morning. ...,Negative,2023-01-15 08:45:00,CommuterX,Twitter,#Traffic #Morning,5.0,10.0,Canada,2023,1,15,8
2,2,2,Just finished an amazing workout! 💪 ...,Positive,2023-01-15 15:45:00,FitnessFan,Instagram,#Fitness #Workout,20.0,40.0,USA,2023,1,15,15
3,3,3,Excited about the upcoming weekend getaway! ...,Positive,2023-01-15 18:20:00,AdventureX,Facebook,#Travel #Adventure,8.0,15.0,UK,2023,1,15,18
4,4,4,Trying out a new recipe for dinner tonight. ...,Neutral,2023-01-15 19:55:00,ChefCook,Instagram,#Cooking #Food,12.0,25.0,Australia,2023,1,15,19


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 732 entries, 0 to 731
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0.1  732 non-null    int64  
 1   Unnamed: 0    732 non-null    int64  
 2   Text          732 non-null    object 
 3   Sentiment     732 non-null    object 
 4   Timestamp     732 non-null    object 
 5   User          732 non-null    object 
 6   Platform      732 non-null    object 
 7   Hashtags      732 non-null    object 
 8   Retweets      732 non-null    float64
 9   Likes         732 non-null    float64
 10  Country       732 non-null    object 
 11  Year          732 non-null    int64  
 12  Month         732 non-null    int64  
 13  Day           732 non-null    int64  
 14  Hour          732 non-null    int64  
dtypes: float64(2), int64(6), object(7)
memory usage: 85.9+ KB


In [5]:
df['Sentiment'].value_counts()

Sentiment
 Positive           44
 Joy                42
 Excitement         32
 Neutral            14
 Contentment        14
                    ..
 Adrenaline          1
 Harmony             1
 ArtisticBurst       1
 Radiance            1
 Elegance            1
Name: count, Length: 279, dtype: int64

In [6]:
import emoji

text_data = df['Text'].tolist()
new_text_data = [ch.strip() for ch in text_data]
new_2_text_data = [emoji.demojize(ch) for ch in new_text_data]
text_data = new_2_text_data



In [16]:
from transformers import pipeline

# Initialize the zero-shot classification pipeline with DeBERTa-v3 model
classifier = pipeline("zero-shot-classification", model="MoritzLaurer/DeBERTa-v3-base-mnli-fever-anli")

def label_data(texts):
    label_mapping = {"positive": 1, "negative": -1, "neutral": 0}
    results = classifier(texts, candidate_labels=["positive", "negative", "neutral"], multi_label=False)
    # Map the verbal labels to numerical labels
    numerical_labels = [label_mapping[result['labels'][0]] for result in results]
    return numerical_labels


labels = label_data(text_data)
print(labels)  


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


[1, -1, 1, 1, 1, 1, -1, 1, -1, -1, 0, -1, 1, 1, 0, 0, 1, 0, 0, -1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, -1, -1, -1, -1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, -1, 1, -1, 1, 1, 1, -1, -1, -1, -1, -1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 0, 1, 1, -1, 1, -1, 1, 1, 1, -1, 1, 1, -1, 1, -1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 0, 1, 0, -1, -1, -1, 1, -1, 1, 1, 1, 1, 0, -1, -1, -1, 1, -1, 1, 1, 1, 1, 0, 0, -1, -1, 1, -1, 1, 1, 1, 0, -1, 0, -1, 1, -1, 1, 1, 1, 1, 0, -1, 0, -1, 1, -1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import numpy as np


# Vectorize words
vectorizer = TfidfVectorizer(max_features=5000)
X_train = vectorizer.fit_transform(text_data)
y_train = np.array(labels)

model = LogisticRegression()
model.fit(X_train, y_train)



LogisticRegression()

In [18]:
from modAL.models import ActiveLearner
import numpy as np

learner = ActiveLearner(
    estimator=LogisticRegression(),
    X_training=X_train, y_training=y_train
)


In [24]:
def user_labeling(samples):
    user_labels = []
    print("Please label the following samples:")
    for sample in samples:
        print(f"Sample: {sample}")
        label = int(input("Enter label (1 for positive, 0 for neutral, -1 for negative): "))
        user_labels.append(label)
    return user_labels

# Active learning loop with a maximum number of iterations
max_iterations = 20  # Set a limit on the number of iterations
batch_size = 5  # Number of samples to label in each iteration
remaining_texts = text_data[len(text_data)//2:]

for iteration in range(max_iterations):
    if len(remaining_texts) == 0:
        print("All data has been labeled.")
        break
    
    X_pool = vectorizer.transform(remaining_texts)
    query_idx, query_instance = learner.query(X_pool, n_instances=batch_size)
   
    low_conf_samples = [remaining_texts[idx] for idx in query_idx]

    new_labels = user_labeling(low_conf_samples)
    
    # Ensure the number of new_labels matches the number of queried samples
    if len(new_labels) != len(query_idx):
        raise ValueError(f"Number of new labels ({len(new_labels)}) does not match number of queried samples ({len(query_idx)})")
    
    learner.teach(X_pool[query_idx], new_labels)
    
    # Remove labeled instances from the pool
    remaining_texts = [remaining_texts[idx] for idx in range(len(remaining_texts)) if idx not in query_idx]

print(f"Active learning completed after {iteration+1} iterations.")

# '''
# for automatic labelling 

# '''
# def automatic_labeling(samples):
#     return label_data(samples)

# # Active learning loop with a maximum number of iterations
# max_iterations = 5  # Set a limit on the number of iterations
# batch_size = 2  # Number of samples to label in each iteration
# remaining_texts = text_data[len(text_data)//2:]

# for iteration in range(max_iterations):
#     if len(remaining_texts) == 0:
#         print("All data has been labeled.")
#         break
    
#     X_pool = vectorizer.transform(remaining_texts)
#     query_idx, query_instance = learner.query(X_pool, n_instances=batch_size)
    
#     # Extract the low confidence samples
#     low_conf_samples = [remaining_texts[idx] for idx in query_idx]
    
#     # Automatically fill in the labels for these samples
#     new_labels = automatic_labeling(low_conf_samples)
    
#     # Ensure the number of new_labels matches the number of queried samples
#     if len(new_labels) != len(query_idx):
#         raise ValueError(f"Number of new labels ({len(new_labels)}) does not match number of queried samples ({len(query_idx)})")
    
#    
#     learner.teach(X_pool[query_idx], new_labels)
    
#     # Remove labeled instances from the pool
#     remaining_texts = [remaining_texts[idx] for idx in range(len(remaining_texts)) if idx not in query_idx]

# print(f"Active learning completed after {iteration+1} iterations.")



Active learning completed after 5 iterations.


In [25]:
# Final model training
final_model = learner.estimator

all_labels = label_data(text_data)


final_labeled_data = pd.DataFrame({'text': text_data, 'label': all_labels})
# Mapping function to convert numerical labels to verbal labels
def numerical_to_verbal(label):
    label_mapping = {-1: "negative", 0: "neutral", 1: "positive"}
    return label_mapping[label]

final_labeled_data['label'] = final_labeled_data['label'].apply(numerical_to_verbal)

final_labeled_data.to_csv('labeled_data.csv', index=False)